In [45]:
%pip install beautifulsoup4
%pip install requests

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Home

In [46]:
# GLOBAL variable
seed_url = 'https://www.wongnai.com/'
frontier_q = [seed_url,'https://cookpad.com/th']
visited_q = []
visited_host = set()
# KU_DOMAIN = 'ku.ac.th'
KEY_WORD = ('วัตถุดิบ','แคลอรี่','อาหาร','เมนู','ของกิน','กับข้าว','รสชาติ','อร่อย','เครื่องเคียง','ของว่าง','เครื่องดื่ม','ขนม')
PAGE_COUNT = 0
WEB_TYPE = ('html')
FILE_TYPE = ('pdf', 'doc', 'xls', 'ppt', 'exe', 'jpg', 'mpg', 'zip', 'xlsx', 'png')
headers = {
    'User-Agent': 'Atipat Pankong',
    'From': 'atipat.pa@ku.th'
}
debug=True

In [47]:
# check backup file
try:
    f = open('pending_queue.txt' ,'r')
    s = f.read()
    if(s):
        url_q = s.splitlines()
        PAGE_COUNT = int(url_q[0])
        frontier_q = url_q[1:]
    if(debug):
        print(PAGE_COUNT)
        print(frontier_q)
except Exception as err:
    print(err)
    print('no backup')
    PAGE_COUNT = 0
    frontier_q = [seed_url]

0
['https://www.wongnai.com/']


In [48]:

# print for debug function.
def show_debug(url,error=False):
    global PAGE_COUNT
    if(not error):
        PAGE_COUNT += 1
        print('page: ',PAGE_COUNT)
    print('site: ',url)
    print('-'*50)

In [49]:
# FIFO queue
def enqueue(links):
    global frontier_q
    for link in links:
        if link not in frontier_q and link not in visited_q and not link.endswith(FILE_TYPE) and not 'mailto:' in link:
            # print('in')
            frontier_q.append(link)
        # else : 
        #     print('out')

def dequeue():
    global frontier_q
    current_url = frontier_q[0]
    frontier_q = frontier_q[1:]
    return current_url

In [50]:
import requests
from requests.exceptions import HTTPError
from urllib.parse import urljoin, urlparse

def get_page(url,require_content_type='text/html',join_set=True):
    global headers
    text = ''
    try:
        if(url.endswith(FILE_TYPE) or 'mailto:' in url): return text
        response = requests.get(url, headers=headers, timeout=2)
        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
        if(debug):
            show_debug(url,error=True)
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
        if(debug):
            show_debug(url,error=True)
    else:
        if(require_content_type in response.headers['Content-Type']):
            # response.encoding = response.apparent_encoding
            text = response.text
            if(sum([1 for x in KEY_WORD if x in text])< 1):
                return ""
            if(join_set):
                parse = urlparse(url)
                visited_host.add(parse.scheme + '://' + parse.netloc)
        else:
            print('"' + response.headers['Content-Type'], '" is wrong content type.')
            if(debug):
                show_debug(url,error=True)
    return text.lower()

In [51]:
import os, codecs
from bs4 import BeautifulSoup
from urllib.parse import unquote

def write_html(url,html_source):
    global WEB_TYPE
    filepath = ''
    filename = 'dummy'
    result = urlparse(url)
    soup = BeautifulSoup(html_source, 'html.parser')
    if(result.path.endswith(WEB_TYPE)):
        filepath = 'html/' + unquote(result.netloc + result.path[:result.path.rfind('/')])
        filename = result.path[result.path.rfind('/')+1:]
    else: 
        filepath = 'html/' + unquote(result.netloc + result.path)

    os.makedirs(filepath, 0o755, exist_ok=True)
    abs_path = filepath + '/' + filename
    if(debug):
        print('write on file directory : ',abs_path)
    f = codecs.open(abs_path ,'w' ,'utf-8' )
    f.write(soup.get_text())
    f.close()

In [52]:
# parser link from html.

def absolute_link(url):
    o = urlparse(url)
    base_url = o.scheme + '://' + o.netloc
    return  base_url + o.path

def link_parser(url):
    html_source = get_page(url)
    if(html_source == ''): return []
    write_html(url,html_source)
    if(debug):
        show_debug(url)
    soup = BeautifulSoup(html_source, 'html.parser')
    all_el = soup.find_all('a',href=True)
    return [absolute_link(urljoin(url,el['href'].strip())) for el in all_el]

In [53]:
# write html > 10000 page.

while PAGE_COUNT < 100 and frontier_q:
    # backup url that was pending in queue.
    try:
        if PAGE_COUNT%100 == 0 :
            f = codecs.open('pending_queue.txt' ,'w' ,'utf-8' )
            f.write(str(PAGE_COUNT) + '\n'+ '\n'.join(frontier_q))
            f.close()
        current_url = dequeue()
        visited_q.append(current_url)
        enqueue(link_parser(current_url))
    except Exception as err:
        print(err)
        pass

write on file directory :  html/www.wongnai.com//dummy
page:  1
site:  https://www.wongnai.com/
--------------------------------------------------
https://www.wongnai.com/
['https://www.wongnai.com/']
out
https://www.wongnai.com/restaurants
['https://www.wongnai.com/']
in
https://www.wongnai.com/restaurants/copper/reviews
['https://www.wongnai.com/']
in
https://www.wongnai.com/restaurants/copper
['https://www.wongnai.com/']
in
https://www.wongnai.com/restaurants/chocolateville/reviews
['https://www.wongnai.com/']
in
https://www.wongnai.com/restaurants/chocolateville
['https://www.wongnai.com/']
in
https://www.wongnai.com/restaurants/leboeufthesteakfriesbistro/reviews
['https://www.wongnai.com/']
in
https://www.wongnai.com/restaurants/leboeufthesteakfriesbistro
['https://www.wongnai.com/']
in
https://www.wongnai.com/restaurants/5290th-%e0%b8%84%e0%b8%a3%e0%b8%b1%e0%b8%a7%e0%b8%94%e0%b8%ad%e0%b8%81%e0%b9%84%e0%b8%a1%e0%b9%89%e0%b8%82%e0%b8%b2%e0%b8%a7-%e0%b8%aa%e0%b8%b2%e0%b8%82%e0%b8%b2

In [ ]:
# write all host in file after run 10000 page successfully.
if(debug):
  print(visited_host)
  f = codecs.open('list_host.txt' ,'w' ,'utf-8' )
  f.write('\n'.join([host for host in visited_host]))
  f.close()

{'https://www.wongnai.com'}


In [ ]:
# find robots.txt in all host and write list of host that have robots.txt or sidemap.
# robots_host = []
# sitemap_host = []
# for host in visited_host:
#     text = get_page(host + '/robots.txt', 'text/plain',False)
#     if(text):
#         robots_host.append(urlparse(host).netloc)
#         if('sitemap:' in text):
#             sitemap_host.append(urlparse(host).netloc)

# f = codecs.open( 'list_robots.txt' , 'w', 'utf-8') 
# f.write('\n'.join(robots_host))
# f.close()

# f = codecs.open('list_sitemap.txt', 'w' , 'utf-8')
# f.write('\n'.join(sitemap_host))
# f.close()